## ライブラリのインポート

In [2]:
import pandas as pd
import glob
from tqdm import tqdm

## データのダウンロード

In [3]:
import pandas as pd


# パスで指定したファイルの一覧をリスト形式で取得. （ここでは一階層下のtestファイル以下）
csv_files = glob.glob('C:\\Users\\rutai\\OneDrive\\Documents\\GitHub\\Data\\*.csv')

#読み込むファイルのリストを表示
for a in csv_files:
    print(a)

#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
    data_list.append(pd.read_csv(file))

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
test_ = pd.concat(data_list, axis=0, sort=True)

C:\Users\rutai\OneDrive\Documents\GitHub\Data\2019_20_35.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2019_35_50.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2019_50_65.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2019_65_80.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2020_20_35.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2020_35_50.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2020_50_65.csv
C:\Users\rutai\OneDrive\Documents\GitHub\Data\2020_65_80.csv


## 特徴量エンジニアリング

### データの作成

In [4]:
for index, data in tqdm(test_.iterrows()):
  """ ageの作成 """
  age_num = data["user_age"][1:-1].split(",")
  test_.loc[index,"age"] = (int(age_num[0])+int(age_num[1]))/2

  """ 価格の作成 """
  price_num = data["price"][1:-1].split(",")
  test_.loc[index,"new_price"] = (int(price_num[0])+int(price_num[1]))/2

  """ genreの作成 """
  genre_list = data["genre_name_all"].split(">>")
  for idx, genre in enumerate(genre_list):
    test_.loc[index,f"genre_{idx}"] = genre_list[idx]

  """ pseudo lon(経度), lat(緯度) の作成"""
  if data["user_region"] == "北海道": # 北海道の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 43.06417,141.34694

  elif data["user_region"] == "東北": # 宮城の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 38.26889,140.87194

  elif data["user_region"] == "関東": # 東京の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 35.68944,139.69167

  elif data["user_region"] == "中部": # 名古屋の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 35.18028,136.90667

  elif data["user_region"] == "近畿": # 大阪の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 34.68639,135.52

  elif data["user_region"] == "中国": # 広島の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 34.39639,132.45944

  elif data["user_region"] == "四国": # 愛媛の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 33.84167,132.76611

  elif data["user_region"] == "九州": # 福岡の緯度経度
    test_.loc[index,f"pseudo_lat"], test_.loc[index,f"pseudo_lon"] = 33.60639,130.41806


43610it [1:44:02,  5.62it/s]

### データの変換

In [ ]:
"""frequency encodeの作成"""
for col in ["use_gender", "user_age", "user_region", "price", "genre_name_all"]:
    enc_nom_1 = (test_.groupby(col).size())/len(test_)
    test_[f'{col}_freq'] = test_[col].apply(lambda x : enc_nom_1[x])
    
"""one-hot encodeの作成"""
test_ = pd.get_dummies(test_)

## メモリの削減

In [ ]:
def reduce_mem_usage(df):
    """ メモリ削減 する関数
    
    Args:
        df(DataFrame) : データベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    # float64 と float32 のデータタイプを float16 に加工
    cols1 = list(df.dtypes[df.dtypes == 'float64'].index) + list(df.dtypes[df.dtypes == 'float32'].index) 
    for col in tqdm(cols1):
        df[col] = df[col].astype(np.float16)

    # int64, int32, int16 のデータタイプを int8 に加工 
    cols2 = list(df.dtypes[df.dtypes == 'int64'].index) + list(df.dtypes[df.dtypes == 'int32'].index)+ list(df.dtypes[df.dtypes == 'int16'].index)
    for col in tqdm(cols2):
        df[col] = df[col].astype(np.int8)

    return df

In [ ]:
test_ = reduce_mem_usage(test_)

## データを保存

In [ ]:
test_.to_csv("data_all.csv", index=False)